## Aula 4 - Agregação com Pandas e Conceitos de Estatística

### Agregação com Pandas

In [ ]:
import pandas as pd
import numpy as np

#### Exemplo de Group By com Pandas

In [ ]:
# Criando um dataframe
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1
grouped = df['data1'].groupby(df['key1'])
# Veja que o resultado retornado é um objeto Pyhton SeriesGroupBy 
grouped

In [ ]:
# Aplicando a métrica Média (mean) sobre esse objeto de agrupamento
# Perceba que a média sobre o objeto SeriesGroupBy vai retornar uma Series como resultado,
#   e o índice será a coluna de agrupamento key1
grouped.mean()

#### Agregação com duas colunas

In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1 e key2
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
# Veja que means é uma Series com índices key1 e key2
means

In [ ]:
# Curiosidade: você pode pivotar essa série com o médoto unstack()
means.unstack()

#### Agregação com vetor do tamanho do índice

In [ ]:
# Considere o dataframe df definido anteriormente e os arrays states e years
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
# Crie um agrupamento da coluna data1 considerando como colunas de agrupamento os arrays
#    states e years
df['data1'].groupby([states, years]).mean()

#### Aplicando a métrica em todas as colunas de dados

In [ ]:
# Agrupe o dataframe df por key1 e extraia a média de todas as colunas
df.groupby('key1').mean()

In [ ]:
# perceba que o objeto retornado pelo agrupamento do dataframe inteiro é 
#   um DataFrameGroupBy 
df.groupby('key1')

In [ ]:
# Agrupe o dataframe inteiro pelas colunas key1 e key2 calculando o valor da média
#    para as outras colunas.
df.groupby(['key1', 'key2']).mean()

#### Mostrando o tamanho dos grupos

In [ ]:
# Mostre o tamanho dos grupos de um agrupamento do dataframe df pelas colunas key1 e key2
df.groupby(['key1', 'key2']).size()

#### Agrupando com uma função

In [ ]:
# Considere o seguinte dataframe
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Adicionando alguns valores NA
people

In [ ]:
# Agrupe o dataframe people pelo tamanho das strings do indice retornando a soma de cada
#    grupo.
people.groupby(len).sum()



#### Aplicando várias métricas ao mesmo tempo

In [ ]:
# Considere o seguinte dataset de gorgetas (tips)
import pandas as pd
import io # Biblioteca com funções de IO (dentre elas de Strings)
import requests #Biblioteca para fazer requisições HTTP
url="https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv"
s=requests.get(url).content
tips=pd.read_csv(io.StringIO(s.decode('utf-8')))
tips.head()

In [ ]:
# Adiciona uma coluna para calcular a porcentage da gorjeta em relação ao valor da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

In [ ]:
# Agrupar por dia da semana (day) e fumante (smoker)
grouped = tips.groupby(['day', 'smoker'])
grouped

In [ ]:
# Pegar um slice (uma fatia) do objeto de agrupamento com apenas a coluna tip_pct
grouped_pct = grouped[['tip_pct']]
grouped_pct

In [ ]:
# Calcular a média deste agrupamento
grouped_pct.agg('mean')

In [ ]:
# Considere a função para retornar o intervalo de variação de um array
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [ ]:
# Escreva numa só sentença a aplicação das métricas média, desvio padrão (std) 
#   e peak_to_peak ao slice do agrupamento criado acima
grouped_pct.agg(['mean', 'std', peak_to_peak])

#### Várias métricas em diferentes colunas

In [ ]:
# Usando um dicionário para mapear as colunas e suas respectivas métricas, aplique
#     à coluna tip_pct as métricas 'min', 'max', 'mean', 'std' e 
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

### Conceitos básicos de Estatística Descritiva

#### Variaveis categóricas

In [ ]:
# Categorias são inferidas a partir dos dados e são não ordenadas
df = pd.DataFrame({'A': list('abca'), 'B': list('bccd')}, dtype="category")
df_cat = df.astype('category')
df_cat.dtypes

In [ ]:
# Também é possível converter os tipos de um dataframe para category
df = pd.DataFrame({'A': list('abca'), 'B': list('bccd')})
df_cat = df.astype('category')
df_cat.dtypes

In [ ]:
# Criando dados categóricos ordenados
from pandas.api.types import CategoricalDtype
s = pd.Series(["a", "b", "c", "a"])
cat_type = CategoricalDtype(categories=["b", "c", "d"], ordered=True)
s_cat = s.astype(cat_type)
s_cat

#### Amostragem

In [ ]:
df = pd.DataFrame({'num_legs': [2, 4, 8, 0],
                   'num_wings': [2, 0, 0, 0],
                   'num_specimen_seen': [10, 2, 1, 8]},
                  index=['falcon', 'dog', 'spider', 'fish'])
df

In [ ]:
# Criar, com o método sample, uma amostra aleatória de 3 elementos com um 
# estado aleatório conhecido. O estado aleatorio é utilizado para assegurar
# reproducibilidade de um experimento.
df['num_legs'].sample(n=3, random_state=10)

In [ ]:
# Criar uma amostra aleatoria com 50% dos elementos, estado aleatório conhecido e 
# com reposição
df.sample(frac=0.5, replace=True, random_state=1)

In [ ]:
# Pode-se fazer um upsample, mas o parâmetro replace deve ser True
df.sample(frac=2, replace=True, random_state=1)

In [ ]:
# Histograma
mm = 168 # média 
sigmam = 5 # desvio padrão
samples = 500
np.random.seed(0)
altura_m = np.random.normal(mm, sigmam, samples).astype(int)

mh = 176 # média
sigmah = 6 # desvio padrão
sample = 500
np.random.seed(1)
altura_h = np.random.normal(mh, sigmah, sample).astype(int)
df_altura = pd.DataFrame({'altura_h': altura_h, 'altura_m': altura_m})
df_altura.head()

In [ ]:
# Criar o histograma dos dados no dataframe df_altura
df_altura.hist()

In [ ]:
# Criar o histograma dos dados no dataframe df_altura especificando a quantidade de bins 
df_altura.hist(bins=20)

#### Mediana

In [ ]:
# Considere o seguinte dataframe  
df = pd.DataFrame({"a":[112, 42, 5, 1, 1, 1, 1, 2, 7,8], 
                   "b":[37, 42, 5424, 311, 5345, 1234, 9098, 1, 5, None],  
                   "c":[210, 416, 114, None, 8, 5, 6, 23, 67, 234],  
                   "d":[144, 23, 1, 45, 890, 1, 234, 43, 12, 62]}) 
df

In [ ]:
# Calcule a mediana dos valores das colunas (axis=0)
df.median(axis = 0, skipna = True) 

#### Quartis

In [ ]:
# Considere o seguinte dataframe do PIB dos municípios
path_pib = '../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx'
df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, 
                       dtype={'Unnamed: 1': str, '2007': str}) 

# Descobrindo porque ocorre um erro ao se tentar ler a coluna 2007 como numero 
df_pib.loc[df_pib['2007'].apply(len) < 4,'2007']
# É porque os valores NA estão representados por ...
# Ler novamente o dataset especificando a string de NA
df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, na_values='...',
                       dtype={'Unnamed: 1': str, '2007': float, '2009': float, '2011': float})

df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                           'Unnamed: 2': 'municipio'}, inplace=True)

In [ ]:
# Calcular os quartis 0.25, .5, .75 e o percentil 0.9 do dataframe do PIB em todos os
#   anos presentes no dataset.
quartis = df_pib.quantile([.25, .5, .75, .9])
quartis

In [ ]:
# Plotar num gráfico de linhas os percentis ao longo dos anos
# quartis.T retorna o dataframe transposto 
quartis.T.plot()

In [ ]:
# Plote o boxplot de cada um dos anos ['2017', '2013', '2011'] do dataset do PIB
df_pib.boxplot(column=['2017', '2013', '2011'])